In [10]:
from time import ctime
import threading
import time

# 定义新的子线程coding
def coding(language):
    print('thread %s is running...' % threading.current_thread().name)
    for i in range(5):
        print('I\'m coding ', language, ' program at ', ctime())

# 定义新的子线程music
def music():
    init_time = time.time()
    print('thread %s is running...' % threading.current_thread().name)
    for i in range(5):
        print('I\'m listening music at ', ctime())
    print(time.time()-init_time)


if __name__ == '__main__':

    print('thread %s is running...' % threading.current_thread().name)
    
    # 定义进程表
    thread_list = []
    
    t1 = threading.Thread(target=coding, name="thread_1", args=('Python',))
    t2 = threading.Thread(target=music, name='thread_2')
    thread_list.append(t1)
    thread_list.append(t2)

    for t in thread_list:
        t.setDaemon(True)  # 设置为守护线程
        t.start()
        t.join()  # 在这个子线程完成运行之前，主线程将一直被阻塞
        print('------------')
        print('thread %s is running...' % threading.current_thread().name)
#     t.join()
    print('thread %s ended.' % threading.current_thread().name)

thread MainThread is running...
thread thread_1 is running...
I'm coding  Python  program at  Tue Mar 26 20:26:33 2019
I'm coding  Python  program at  Tue Mar 26 20:26:33 2019
I'm coding  Python  program at  Tue Mar 26 20:26:33 2019
I'm coding  Python  program at  Tue Mar 26 20:26:33 2019
I'm coding  Python  program at  Tue Mar 26 20:26:33 2019
------------
thread MainThread is running...
thread thread_2 is running...
I'm listening music at  Tue Mar 26 20:26:33 2019
I'm listening music at  Tue Mar 26 20:26:33 2019
I'm listening music at  Tue Mar 26 20:26:33 2019
I'm listening music at  Tue Mar 26 20:26:33 2019
I'm listening music at  Tue Mar 26 20:26:33 2019
0.002607583999633789
------------
thread MainThread is running...
thread MainThread ended.


In [3]:
import time, threading

# 新线程执行的代码:
def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)
    print('----')

print('thread %s is running...' % threading.current_thread().name)
t = threading.Thread(target=loop, name='LoopThread')
t.start()
t.join()
print('thread %s ended.' % threading.current_thread().name)

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended.
----
thread MainThread ended.


#### Lock
多线程和多进程最大的不同在于，多进程中，同一个变量，各自有一份拷贝存在于每个进程中，互不影响，而多线程中，所有变量都由所有线程共享，所以，任何一个变量都可以被任何一个线程修改，因此，线程之间共享数据最大的危险在于多个线程同时改一个变量，把内容给改乱了。  
来看看多个线程同时操作一个变量怎么把内容给改乱了：  

我们定义了一个共享变量balance，初始值为0，并且启动两个线程，先存后取，理论上结果应该为0，但是，由于线程的调度是由操作系统决定的，当t1、t2交替执行时，只要循环次数足够多，balance的结果就不一定是0了。

In [18]:
import time, threading

# 假定这是你的银行存款:
balance = 0

def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n

def run_thread(n):
    for i in range(10000000):
        change_it(n)

t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

37


两个线程同时一存一取，就可能导致余额不对，你肯定不希望你的银行存款莫名其妙地变成了负数，所以，我们必须确保一个线程在修改balance的时候，别的线程一定不能改。  
如果我们要确保balance计算正确，就要给change_it()上一把锁，当某个线程开始执行change_it()时，我们说，该线程因为获得了锁，因此其他线程不能同时执行change_it()，只能等待，直到锁被释放后，获得该锁以后才能改。由于锁只有一个，无论多少线程，同一时刻最多只有一个线程持有该锁，所以，不会造成修改的冲突。创建一个锁就是通过threading.Lock()来实现：



In [21]:
from threading import Lock

balance = 0
lock = Lock()
def run_thread(n):
    for i in range(1000000):
        # 先要获取锁:
        lock.acquire()
        try:
            # 放心地改吧:
            change_it(n)
        finally:
            # 改完了一定要释放锁:
            lock.release()

def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n

if __name__ == "__main__":
    t1 = threading.Thread(target=run_thread, args=(5,))
    t2 = threading.Thread(target=run_thread, args=(8,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print(balance)

0
